In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import yfinance as yf
import pandas as pd
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from typing import Dict
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np
from scipy.stats import linregress
from ta import add_all_ta_features

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.common.AssetDataPolars import AssetDataPolars
from src.mathTools.CurveAnalysis import CurveAnalysis
from src.predictionModule.FourierML import FourierML
from src.common.YamlTickerInOut import YamlTickerInOut
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPandas as DFTO


In [3]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_swiss_over20years")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker]= AssetDataService.to_polars(asset)

In [7]:
aIRM = assetspl["VETN.SW"]
sharepriceArr = aIRM.shareprice
pricesArray = aIRM.shareprice["Close"]
aidx = pricesArray.len() - 10
m=24
pastPrices = pricesArray.slice(aidx-m * 21-1, m * 21 +1).to_numpy()
futurePrices = pricesArray.slice((aidx+1),5).to_numpy()

In [9]:
print(np.any(np.isnan(aIRM.shareprice["Close"].to_numpy())))
print(np.any(np.isnan(aIRM.shareprice["Adj Close"].to_numpy())))
print(np.any(aIRM.shareprice["Close"].to_numpy()<=-1.0))
print(np.any((aIRM.shareprice["Adj Close"].to_numpy()<=-1.0)))

False
False
False
True


In [8]:
ohlc = aIRM.shareprice.slice(aidx-m * 21-1, m * 21 +1)
print(ohlc.columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


In [9]:
tadata = add_all_ta_features(ohlc.to_pandas(), open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
#print(tadata['Date'].head())
#print(tadata.head())
print(type(tadata))
print(tadata.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em',
       'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi',
       'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_atr', 'volatility_ui',
       'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 't

In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns_to_minmax = [
'Volume','volume_adi','volume_obv','volume_cmf','volume_fi','volume_em','volume_sma_em','volume_vpt','volume_mfi','volume_nvi',
'volatility_bbp','volatility_kcw','volatility_kcp','volatility_dcw','volatility_dcp','volatility_atr','volatility_ui','trend_macd','trend_macd_signal',
'trend_macd_diff','trend_vortex_ind_neg','trend_vortex_ind_diff','trend_trix','trend_mass_index','trend_dpo','trend_kst','trend_stc','trend_adx','trend_adx_pos','trend_adx_neg',
'trend_cci','trend_aroon_up','trend_aroon_down','trend_aroon_ind','momentum_stoch_rsi','momentum_stoch_rsi_k','momentum_stoch_rsi_d','momentum_tsi',
'momentum_uo','momentum_stoch','momentum_stoch_signal','momentum_wr','momentum_ao','momentum_roc','momentum_ppo','momentum_ppo_signal',
'momentum_ppo_hist', 'volatility_bbhi','others_dlr','trend_psar_down_indicator','volatility_bbw','trend_kst_sig','trend_visual_ichimoku_b',
'others_dr','volatility_kcli','volatility_bbli','trend_vortex_ind_pos','trend_psar_up_indicator','momentum_pvo_hist',
'volatility_kchi','trend_visual_ichimoku_a','momentum_pvo_signal','momentum_rsi','trend_kst_diff','momentum_pvo', 'others_cr',
]
columns_to_keep = ['Open','High','Low','Close', 'trend_sma_slow', 'trend_ichimoku_conv', 'volatility_bbl', 'volatility_bbm', 
 'volatility_dcm', 'volatility_kcc', 'trend_psar_down', 'trend_sma_fast', 
 'volatility_kch', 'trend_ichimoku_base', 'volume_vwap', 'trend_ichimoku_a', 
 'trend_ema_slow', 'volatility_kcl', 'momentum_kama', 'trend_ema_fast', 'volatility_dcl', 'trend_psar_up', 
 'volatility_dch', 'trend_ichimoku_b', 'volatility_bbh']

columns_to_del = ['Adj Close', 'Date']

scaler = MinMaxScaler()
tadata_minmax = scaler.fit_transform(tadata[columns_to_minmax])
tadata_minmax = pd.DataFrame(tadata_minmax)
tadata_minmax.columns=columns_to_minmax

tadata_ideal = tadata[columns_to_keep]


117.48

kill date

calc diff (adj close to close) then minaxscale

minnmaxscale all vols  except volume_vwap, 

leave volatility_bbw, volatility_bbh, volatility_bbl

minnmax volatility_bbp

leave volatility_bbhi volatility_bbli volatility_kcc, volatility_kch, volatility_kcl, 

minmax volatility_kcw,  volatility_kcp

leave volatility_kchi, volatility_kcli, volatility_dcl, volatility_dch, volatility_dcm

minmax volatility_dcw, volatility_dcp, volatility_atr, volatility_ui, trend_macd, trend_macd_signal, trend_macd_diff,

leave trend_sma_fast, trend_sma_slow, trend_ema_fast, trend_ema_slow, 

minmax trend_vortex_ind_neg, trend_vortex_ind_diff, upto trend_kst_diff, 

leave trend_ichimoku_conv, trend_ichimoku_base, trend_ichimoku_a, trend_ichimoku_b, 

minmax trend_stc, upto trend_cci,

leave trend_visual_ichimoku_a, trend_visual_ichimoku_b,

minmax trend_aroon_up, upto trend_aroon_ind, 

leave trend_psar_up, trend_psar_down,  trend_psar_up_indicator, trend_psar_down_indicator

minmax momentum_stoch_rsi, upto momentum_pvo_hst,

leave momentum_kama upto others_cr



In [26]:
div = (tadata["Close"]/tadata["Adj Close"])
div.head()

0    1.078687
1    1.078687
2    1.078687
3    1.078687
4    1.078687
dtype: float64

In [7]:
pr = tadata.iloc[5000, :].values
print(type(pr.tolist()))

<class 'list'>


In [8]:
aIRM.shareprice.row(5000)

(datetime.datetime(2015, 12, 11, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
 26.579999923706055,
 26.84000015258789,
 26.479999542236328,
 26.75,
 15.920913696289062,
 1365500)

In [19]:
aAdjC = aIRM.adjClosePrice["AdjClose"]

aAdjC.slice(5000, 20).to_numpy()

aAdjClist = aAdjC.to_list()
print(aAdjClist)
print([0] + aAdjClist[1:5] + [0])

[1.219476342201233, 1.2289297580718994, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.2100236415863037, 1.1911171674728394, 1.0965838432312012, 1.0965838432312012, 1.0965838432312012, 1.115490436553955, 1.1533035039901733, 1.1911171674728394, 1.1722102165222168, 1.2005703449249268, 1.2100236415863037, 1.1911171674728394, 1.1911171674728394, 1.1343965530395508, 1.1438500881195068, 1.077676773071289, 1.0965838432312012, 1.1343965530395508, 1.1343965530395508, 1.0965838432312012, 1.0965838432312012, 1.0965838432312012, 1.1533035039901733, 1.0965838432312012, 1.115490436553955, 1.077676773071289, 1.0398640632629395, 1.0398640632629395, 1.077676773071289, 1.0965838432312012, 1.1533035039901733, 1.1722102165222168, 1.1722102165222168, 1.1722102165222168, 1.1722102165222168, 1.1533035039901733, 1.1249431371688843, 1.0965838432312012, 1.1343965530395508, 1.115490436553955, 1.1060